# Analyse du Trafic Maritime - Corrigé

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
df = pd.read_csv('../data/raw/ais_data.csv')

## 1. Exploration & Nettoyage des Données

In [ ]:
print("Colonnes originales:")
print(df.columns)
print("
Informations sur les données:")
df.info()

On supprime les colonnes qui ne sont pas utiles (`Unnamed: 0`) ou qui ont trop de valeurs manquantes (`heading`, `draught`).

In [ ]:
df.drop(columns=['Unnamed: 0', 'heading', 'draught'], inplace=True)

On supprime ensuite les lignes où il manque des valeurs pour les autres colonnes.

In [ ]:
df.dropna(inplace=True)
print("Vérification des valeurs manquantes après nettoyage:")
print(df.isnull().sum())

## 2. Visualisation des Données

In [ ]:
plt.figure(figsize=(12, 8))
sns.countplot(y='shiptype', data=df, order = df['shiptype'].value_counts().index)
plt.title('Distribution des types de navires')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['sog'], bins=30, kde=True)
plt.title('Distribution de la vitesse des navires (SOG)')
plt.show()

## 3. Modélisation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

La colonne `shiptype` est textuelle, il faut l'encoder en valeurs numériques pour que le modèle puisse l'utiliser.

In [ ]:
le = LabelEncoder()
df['shiptype_encoded'] = le.fit_transform(df['shiptype'])

On sélectionne nos caractéristiques (features) et notre cible (target) et on divise les données.

In [ ]:
features = ['sog', 'cog', 'width', 'length']
target = 'shiptype_encoded'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

On entraîne le modèle.

In [ ]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

## 4. Évaluation du Modèle

In [ ]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}
')

# Pour afficher le rapport avec les noms de classes originaux
y_test_decoded = le.inverse_transform(y_test)
y_pred_decoded = le.inverse_transform(y_pred)
print(classification_report(y_test_decoded, y_pred_decoded, zero_division=0))